## Challenge
In this lab, the objective is to identify the customers who were active in both May and June, and how did their activity differ between months. To achieve this, follow these steps:

In [1]:
import mysql.connector
from sqlalchemy import create_engine
from getpass import getpass
import pandas as pd

1. Establish a connection between Python and the Sakila database.

In [2]:
#set my password
my_password = getpass()

#connect to sql
cnx = mysql.connector.connect(user = "root", 
                              password=my_password,
                              host = "localhost",
                              database = "sakila")
if cnx.is_connected()==True:
    print("Succefully Connected!")
else:
    print("Try again!")

#define an object to interact with SQL
cursor = cnx.cursor()

Succefully Connected!


2. Write a Python function called `rentals_month` that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

	- `engine`: an object representing the database connection engine to be used to establish a connection to the Sakila database.
	- `month`: an integer representing the month for which rental data is to be retrieved.
	- `year`: an integer representing the year for which rental data is to be retrieved.

	The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [3]:
def rentals_month(engine, mon, ye):

#execute a SQL query to retrieve the rental data for the specified month and year from the rental 
# table in the Sakila database, and return it as a pandas DataFrame
     query = (f"""SELECT * FROM rental
          WHERE month(rental_date) = {mon}
          AND
          year(rental_date) = {ye};""")

     cursor.execute(query)
     columns_names = [element[0] for element in cursor.description]
     result = pd.DataFrame(cursor.fetchall(), columns=columns_names)
     return  result

In [4]:
month = 5
year = 2005
df_rentals_month = rentals_month(cnx, month, year)
df_rentals_month.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


3. Develop a Python function called `rental_count_month` that takes the DataFrame provided by `rentals_month` as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year. 

	The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".
	
	*Hint: Consider making use of pandas [groupby()

In [5]:

def rental_count_month(df, mon, ye):
  
    #filter the data by month and year
    filtered_df = df[(df['rental_date'].dt.month == mon) & 
                (df['rental_date'].dt.year == ye)]
    
    #count the rentals by customer_id
    rent_count = filtered_df['customer_id'].value_counts().reset_index()
    rent_count.columns = ['customer_id', 'rentals_{mon}_{ye}']
    rent_count.sort_values(by='customer_id', inplace=True)
    rent_count.reset_index()

    return rent_count

In [6]:
#remember that only the clients selected in the previous step will be considered here!
mon = 5
ye = 2005
rental_count_month(df_rentals_month,mon,ye)

,customer_id,rentals_{mon}_{ye}
292,1,2
422,2,1
243,3,2
129,5,3
153,6,3
...,...,...
60,594,4
496,595,1
9,596,6
287,597,2


4. Create a Python function called `compare_rentals` that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. 
The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [7]:

def compare_rentals(df_1, df_2):
    merged_df = pd.merge (df_1, df_2, on='customer_id', how='inner')


    return merged_df

In [8]:
m1 = 5
m2 = 6
y1 = 2005
y2 = 2005

df1 = rentals_month(cnx,m1,y1)
df_final1 = rental_count_month(df1, m1,y1)

df2 = rentals_month(cnx,m2,y2)
df_final2 = rental_count_month(df2, m2,y2)

#diference = df_final1['rentals_{mon}_{ye}'] - df_final2['rentals_{mon}_{ye}']

compare_rentals(df_final1, df_final2)

,customer_id,rentals_{mon}_{ye}_x,rentals_{mon}_{ye}_y
0,1,2,7
1,2,1,1
2,3,2,4
3,5,3,5
4,6,3,4
...,...,...,...
507,594,4,6
508,595,1,2
509,596,6,2
510,597,2,3
